In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, Normalizer, MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn import svm, metrics
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
import time
%matplotlib inline

In [2]:
# Load data
train_base = pd.read_csv("./data/train/base_train_sum.csv", encoding="gbk")
train_knowledge = pd.read_csv("./data/train/knowledge_train_sum.csv", encoding="gbk")
train_money = pd.read_csv("./data/train/money_report_train_sum.csv", encoding="gbk")
train_year = pd.read_csv("./data/train/year_report_train_sum.csv", encoding="gbk")
verify_base = pd.read_csv("./data/verify/base_verify1.csv", encoding="gbk")
verify_money = pd.read_csv("./data/verify/money_information_verify1.csv", encoding="gbk")
verify_knowledge = pd.read_csv("./data/verify/paient_information_verify1.csv", encoding="gbk")
verify_year = pd.read_csv("./data/verify/year_report_verify1.csv", encoding="gbk")

In [3]:
train_base

,ID,注册时间,注册资本,行业,区域,企业类型,控制人类型,控制人持股比例,flag
0,28,2007.0,2050.0,交通运输业,福建,农民专业合作社,企业法人,NaN,1.0
1,230,2008.0,3360.0,服务业,广东,农民专业合作社,企业法人,1.00,1.0
2,429,2005.0,9670.0,工业,江西,集体所有制企业,自然人,0.75,1.0
3,693,2011.0,8360.0,社区服务,山东,股份有限公司,企业法人,0.98,1.0
4,727,2001.0,8720.0,零售业,江西,股份有限公司,企业法人,0.54,1.0
...,...,...,...,...,...,...,...,...,...
15045,5978029,2014.0,460.0,交通运输业,广西,农民专业合作社,企业法人,0.71,NaN
15046,5978030,2004.0,1140.0,商业服务业,湖南,有限责任公司,自然人,0.95,NaN
15047,5978031,2008.0,3290.0,交通运输业,湖南,集体所有制企业,自然人,0.63,NaN
15048,5978032,2009.0,6060.0,工业,福建,农民专业合作社,企业法人,0.51,NaN


In [4]:
def merge_base_knowledge():
    df_train = pd.merge(train_knowledge, train_base, on='ID')
    df_verify = pd.merge(verify_knowledge, verify_base, on='ID')
    return df_train, df_verify
train_base_knowledge, verify_base_knowledge = merge_base_knowledge()

In [5]:
print(train_base_knowledge.shape)
train_base_knowledge

(15050, 12)


,ID,专利,商标,著作权,注册时间,注册资本,行业,区域,企业类型,控制人类型,控制人持股比例,flag
0,28,0.0,1.0,1.0,2007.0,2050.0,交通运输业,福建,农民专业合作社,企业法人,NaN,1.0
1,230,0.0,0.0,0.0,2008.0,3360.0,服务业,广东,农民专业合作社,企业法人,1.00,1.0
2,429,1.0,0.0,0.0,2005.0,9670.0,工业,江西,集体所有制企业,自然人,0.75,1.0
3,693,0.0,0.0,0.0,2011.0,8360.0,社区服务,山东,股份有限公司,企业法人,0.98,1.0
4,727,0.0,0.0,0.0,2001.0,8720.0,零售业,江西,股份有限公司,企业法人,0.54,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
15045,5978029,1.0,1.0,1.0,2014.0,460.0,交通运输业,广西,农民专业合作社,企业法人,0.71,NaN
15046,5978030,0.0,0.0,0.0,2004.0,1140.0,商业服务业,湖南,有限责任公司,自然人,0.95,NaN
15047,5978031,0.0,1.0,1.0,2008.0,3290.0,交通运输业,湖南,集体所有制企业,自然人,0.63,NaN
15048,5978032,1.0,0.0,1.0,2009.0,6060.0,工业,福建,农民专业合作社,企业法人,0.51,NaN


### 观察发现验证集中多了个***控制人ID***标签，没什么用，删除之 

In [6]:
print(verify_base_knowledge.shape)
verify_base_knowledge.drop('控制人ID', axis=1, inplace=True)
verify_base_knowledge

(30884, 13)


,ID,专利,商标,著作权,注册时间,注册资本,行业,区域,企业类型,控制人类型,控制人持股比例,flag
0,1500001.0,0.0,0.0,0.0,2010.0,6680.0,商业服务业,山东,股份有限公司,自然人,0.89,1.0
1,1500321.0,0.0,0.0,0.0,2001.0,9330.0,商业服务业,广东,股份有限公司,企业法人,0.72,1.0
2,1500395.0,1.0,1.0,0.0,2003.0,8670.0,交通运输业,广西,农民专业合作社,企业法人,0.60,1.0
3,1500614.0,0.0,0.0,0.0,2001.0,7730.0,工业,江西,集体所有制企业,企业法人,0.58,1.0
4,1501057.0,0.0,1.0,1.0,2002.0,6840.0,工业,福建,农民专业合作社,自然人,0.84,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
30879,5999996.0,1.0,1.0,0.0,2011.0,2170.0,零售业,湖北,农民专业合作社,自然人,0.93,0.0
30880,5999997.0,1.0,1.0,0.0,2013.0,9030.0,交通运输业,福建,集体所有制企业,企业法人,0.73,0.0
30881,5999998.0,0.0,0.0,0.0,2014.0,4510.0,服务业,湖南,股份有限公司,自然人,0.64,0.0
30882,5999999.0,1.0,1.0,1.0,2014.0,9130.0,交通运输业,福建,股份有限公司,自然人,0.80,0.0


### 观察发现money和year的数据中，有一些year数据缺失，但是这些数据都是一大堆一起出现的，使用bfill填充之

In [7]:
def merge_money_year(): 
    train_money.loc[:,'year'] = train_money['year'].fillna(method='bfill')
    train_year.loc[:,'year'] = train_year['year'].fillna(method='bfill')
    df_train = pd.merge(train_money, train_year, on=['ID', 'year'])
    
    verify_money.loc[:,'year'] = verify_money['year'].fillna(method='bfill')
    verify_year.loc[:,'year'] = verify_year['year'].fillna(method='bfill')
    df_verify = pd.merge(verify_money, verify_year, on=['ID', 'year'])
    return df_train, df_verify
train_money_year, verify_money_year = merge_money_year()
train_money_year

,ID,year,债权融资额度,债权融资成本,股权融资额度,股权融资成本,内部融资和贸易融资额度,内部融资和贸易融资成本,项目融资和政策融资额度,项目融资和政策融资成本,从业人数,资产总额,负债总额,营业总收入,主营业务收入,利润总额,净利润,纳税总额,所有者权益合计
0,28,2015.0,0.0,0.0,0.00,0.0000,21648.0,1298.880,0.0,0.000,794.0,16400.0,28700.0,72160.0,28864.0,7216.0,-7216.0,0.0,-12300.0
1,230,2015.0,0.0,0.0,0.00,0.0000,0.0,0.000,470.4,28.224,485.0,23520.0,10080.0,115248.0,57624.0,57624.0,-11524.8,0.0,13440.0
2,429,2015.0,19340.0,1547.2,0.00,0.0000,0.0,0.000,0.0,0.000,136.0,193400.0,183730.0,502840.0,351988.0,NaN,-50284.0,0.0,9670.0
3,693,2015.0,0.0,0.0,0.00,0.0000,0.0,0.000,5350.4,321.024,534.0,133760.0,125400.0,655424.0,262169.6,196627.2,-65542.4,0.0,8360.0
4,727,2015.0,0.0,0.0,32229.12,1289.1648,0.0,0.000,0.0,0.000,375.0,366240.0,536280.0,402864.0,282004.8,161145.6,-40286.4,0.0,-170040.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45145,5978029,2017.0,0.0,0.0,0.00,0.0000,1117.8,67.068,0.0,0.000,265.0,1380.0,1840.0,3726.0,1863.0,745.2,372.6,2235.6,-460.0
45146,5978030,2017.0,0.0,0.0,0.00,0.0000,0.0,0.000,2371.2,142.272,1000.0,59280.0,58140.0,260832.0,208665.6,78249.6,-26083.2,0.0,1140.0
45147,5978031,2017.0,0.0,0.0,0.00,0.0000,0.0,0.000,2664.9,159.894,357.0,88830.0,171080.0,106596.0,42638.4,10659.6,-10659.6,0.0,-82250.0
45148,5978032,2017.0,0.0,0.0,0.00,0.0000,0.0,0.000,2908.8,174.528,572.0,145440.0,209070.0,421776.0,253065.6,168710.4,-42177.6,0.0,-63630.0


### 对于money和year两个csv中的其他缺失数据，感觉使用均值填充较为合理

In [8]:
for column in list(train_money_year.columns[train_money_year.isnull().sum() > 0]):
    mean_val = train_money_year[column].mean()
    train_money_year[column].fillna(mean_val, inplace=True)
for column in list(verify_money_year.columns[verify_money_year.isnull().sum() > 0]):
    mean_val = verify_money_year[column].mean()
    verify_money_year[column].fillna(mean_val, inplace=True)

### 按ID排序，方便下面汇总某一个ID3年的平均数据

In [9]:
train_money_year.sort_values('ID', inplace = True)
verify_money_year.sort_values('ID', inplace = True)

In [10]:
train_money_year.isna().any()

ID             False
year           False
债权融资额度         False
债权融资成本         False
股权融资额度         False
股权融资成本         False
内部融资和贸易融资额度    False
内部融资和贸易融资成本    False
项目融资和政策融资额度    False
项目融资和政策融资成本    False
从业人数           False
资产总额           False
负债总额           False
营业总收入          False
主营业务收入         False
利润总额           False
净利润            False
纳税总额           False
所有者权益合计        False
dtype: bool

In [11]:
verify_money_year

,ID,year,债权融资额度,债权融资成本,股权融资额度,股权融资成本,内部融资和贸易融资额度,内部融资和贸易融资成本,项目融资和政策融资额度,项目融资和政策融资成本,从业人数,资产总额,负债总额,营业总收入,主营业务收入,利润总额,净利润,纳税总额,所有者权益合计
0,1500001,2015.0,0.0,0.00,0.0,0.000,0.0,0.000,334.0,20.040,939.0,33400.0,13360.0,150300.0,120240.0,30060.0,-15030.000000,0.0,20040.0
143,1500001,2016.0,0.0,0.00,0.0,0.000,84168.0,5050.080,0.0,0.000,977.0,200400.0,96860.0,280560.0,168336.0,112224.0,-28056.000000,0.0,103540.0
286,1500001,2017.0,0.0,0.00,0.0,0.000,91182.0,5470.920,0.0,0.000,744.0,233800.0,454240.0,303940.0,243152.0,151970.0,-30394.000000,0.0,-220440.0
287,1500321,2017.0,0.0,0.00,0.0,0.000,394099.2,23645.952,0.0,0.000,942.0,298560.0,578460.0,1313664.0,788198.4,394099.2,-131366.400000,0.0,-279900.0
1,1500321,2015.0,0.0,0.00,0.0,0.000,721022.4,43261.344,0.0,0.000,228.0,522480.0,256575.0,2403408.0,961363.2,240340.8,27214.686666,0.0,265905.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48923,5999998,2015.0,0.0,0.00,0.0,0.000,0.0,0.000,135.3,8.118,930.0,13530.0,18040.0,4059.0,2841.3,1623.6,811.800000,2435.4,-4510.0
92649,5999998,2017.0,0.0,0.00,0.0,0.000,18265.5,1095.930,0.0,0.000,841.0,67650.0,63140.0,60885.0,30442.5,30442.5,-6088.500000,0.0,4510.0
70787,5999999,2016.0,7304.0,584.32,0.0,0.000,0.0,0.000,0.0,0.000,933.0,73040.0,95865.0,51128.0,25564.0,5112.8,0.000000,10225.6,-22825.0
48924,5999999,2015.0,0.0,0.00,6025.8,241.032,0.0,0.000,0.0,0.000,885.0,27390.0,9130.0,60258.0,42180.6,30129.0,6025.800000,24103.2,18260.0


### ~~下面这个做均值的函数运行很耗时间，不知道有什么方法能改进不~~
### 现在已经改进了，从100+秒变成只要0.5秒

In [12]:
train_mean = train_money_year.groupby('ID').mean().reset_index()
verify_mean = verify_money_year.groupby('ID').mean().reset_index()

In [13]:
def merge_all():
    df_train = pd.merge(train_mean, train_base_knowledge, on='ID')
    df_verify = pd.merge(verify_mean, verify_base_knowledge, on='ID')
    return df_train, df_verify

In [14]:
df_train, df_verify = merge_all()

### 合并之后shape从15050变成了15048，说明有两个ID不匹配被丢弃了

In [15]:
print(df_train.shape)
df_train

(15048, 30)


,ID,year,债权融资额度,债权融资成本,股权融资额度,股权融资成本,内部融资和贸易融资额度,内部融资和贸易融资成本,项目融资和政策融资额度,项目融资和政策融资成本,...,商标,著作权,注册时间,注册资本,行业,区域,企业类型,控制人类型,控制人持股比例,flag
0,28,2016.0,0.000000,0.000000,12710.000000,508.400000,7216.0,432.960000,0.000000,0.000,...,1.0,1.0,2007.0,2050.0,交通运输业,福建,农民专业合作社,企业法人,NaN,1.0
1,230,2016.0,0.000000,0.000000,1008.000000,40.320000,15590.4,935.424000,156.800000,9.408,...,0.0,0.0,2008.0,3360.0,服务业,广东,农民专业合作社,企业法人,1.00,1.0
2,429,2016.0,6446.666667,515.733333,35321.286667,1412.851467,0.0,0.000000,0.000000,0.000,...,0.0,0.0,2005.0,9670.0,工业,江西,集体所有制企业,自然人,0.75,1.0
3,693,2016.0,0.000000,0.000000,10700.800000,428.032000,18392.0,1103.520000,1783.466667,107.008,...,0.0,0.0,2011.0,8360.0,社区服务,山东,股份有限公司,企业法人,0.98,1.0
4,727,2016.0,0.000000,0.000000,44227.840000,1769.113600,48396.0,2903.760000,0.000000,0.000,...,0.0,0.0,2001.0,8720.0,零售业,江西,股份有限公司,企业法人,0.54,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15043,5978029,2016.0,0.000000,0.000000,0.000000,69.681492,1200.6,72.036000,23.000000,1.380,...,1.0,1.0,2014.0,460.0,交通运输业,广西,农民专业合作社,企业法人,0.71,NaN
15044,5978030,2016.0,0.000000,0.000000,0.000000,0.000000,2633.4,683.727883,1155.200000,69.312,...,0.0,0.0,2004.0,1140.0,商业服务业,湖南,有限责任公司,自然人,0.95,NaN
15045,5978031,2016.0,0.000000,0.000000,8558.386667,342.335467,0.0,0.000000,888.300000,53.298,...,1.0,1.0,2008.0,3290.0,交通运输业,湖南,集体所有制企业,自然人,0.63,NaN
15046,5978032,2016.0,4242.000000,339.360000,8508.240000,340.329600,0.0,0.000000,969.600000,58.176,...,0.0,1.0,2009.0,6060.0,工业,福建,农民专业合作社,企业法人,0.51,NaN


### 把数据除了flag之外的na都填满

In [16]:
values = {'注册时间': int(df_train['注册时间'].mean()), 
          '注册资本': int(df_train['注册资本'].mean()), 
          '控制人持股比例': df_train['控制人持股比例'].mean(),
          '行业': 'other',
          '区域': 'other',
          '企业类型': 'other',
          '控制人类型': 'other',
          '专利': 0,
          '商标': 0,
          '著作权': 0,}
values2 = {'注册时间': int(df_verify['注册时间'].mean()), 
          '注册资本': int(df_verify['注册资本'].mean()), 
          '控制人持股比例': df_verify['控制人持股比例'].mean(),
          '行业': 'other',
          '区域': 'other',
          '企业类型': 'other',
          '控制人类型': 'other',
          '专利': 0,
          '商标': 0,
          '著作权': 0,}
df_train.fillna(value=values, inplace=True)
df_verify.fillna(value=values2, inplace=True)

In [17]:
df_train.isnull().sum()

ID                0
year              0
债权融资额度            0
债权融资成本            0
股权融资额度            0
股权融资成本            0
内部融资和贸易融资额度       0
内部融资和贸易融资成本       0
项目融资和政策融资额度       0
项目融资和政策融资成本       0
从业人数              0
资产总额              0
负债总额              0
营业总收入             0
主营业务收入            0
利润总额              0
净利润               0
纳税总额              0
所有者权益合计           0
专利                0
商标                0
著作权               0
注册时间              0
注册资本              0
行业                0
区域                0
企业类型              0
控制人类型             0
控制人持股比例           0
flag           9977
dtype: int64

### 这里把verify的na都填满了（除flag），发现df_verify.flag也有na, 直接删除df_verify.flag=na的数据

In [18]:
df_verify.isnull().sum()

ID               0
year             0
债权融资额度           0
债权融资成本           0
股权融资额度           0
股权融资成本           0
内部融资和贸易融资额度      0
内部融资和贸易融资成本      0
项目融资和政策融资额度      0
项目融资和政策融资成本      0
从业人数             0
资产总额             0
负债总额             0
营业总收入            0
主营业务收入           0
利润总额             0
净利润              0
纳税总额             0
所有者权益合计          0
专利               0
商标               0
著作权              0
注册时间             0
注册资本             0
行业               0
区域               0
企业类型             0
控制人类型            0
控制人持股比例          0
flag           306
dtype: int64

In [19]:
df_verify.dropna(inplace=True)

In [20]:
# 从30884变成了30577
df_verify.shape

(30577, 30)

### 接下来处理中文数据问题

In [21]:
encoder = OneHotEncoder(sparse = False)
X_train = df_train.to_numpy()
X_verify = df_verify.to_numpy()

In [22]:
Y_train = X_train[:, -1]
X_train = np.delete(X_train, -1, axis = 1)
Y_verify = X_verify[:, -1]
X_verify = np.delete(X_verify, -1, axis = 1)

In [23]:
X_verify[0]

array([1500001, 2016.0, 0.0, 0.0, 0.0, 0.0, 58450.0, 3507.0,
       111.33333333333333, 6.68, 886.6666666666666, 155866.66666666666,
       188153.33333333334, 244933.33333333334, 177242.66666666666,
       98084.66666666667, -24493.333333333332, 0.0, -32286.666666666668,
       0.0, 0.0, 0.0, 2010.0, 6680.0, '商业服务业', '山东', '股份有限公司', '自然人',
       0.89], dtype=object)

In [24]:
zh_train = X_train[:, -5:-1]
X_train = np.hstack((X_train[:, :-5], X_train[:, -1].reshape(-1, 1))).astype('float32')
zh_verify = X_verify[:, -5:-1]
X_verify = np.hstack((X_verify[:, :-5], X_verify[:, -1].reshape(-1, 1))).astype('float32')

In [25]:
encoder.fit(zh_train)
ans_train = encoder.transform(zh_train)
ans_verify = encoder.transform(zh_verify)

In [26]:
print(X_train.shape,ans_train.shape )

(15048, 25) (15048, 24)


In [27]:
X_train = np.hstack((X_train, ans_train)).astype('float32')
X_verify = np.hstack((X_verify, ans_verify)).astype('float32')

In [28]:
X_verify.shape

(30577, 49)

### 这里位置已经把所有表都合并了，并且除了flag之外都没有na了，接下来先做正则化在根据方差选择lable

### 我发现先做聚类，比先做normalizer效果好。先做normalizer会把有lable的数据都分成不同的类 

In [29]:
# 聚类
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans.fit(X_train)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [30]:
for i in range(len(kmeans.labels_)):
    if kmeans.labels_[i] == 1:
        Y_train[i] = 0
    else:
        Y_train[i] = 1

In [31]:
Y_train

array([1, 1, 1, ..., 0, 0, 0], dtype=object)

In [32]:
# trainx = Normalizer().fit_transform(X_train)
# verifyx = Normalizer().fit_transform(X_verify)
trainx = MinMaxScaler().fit_transform(X_train)
verifyx = MinMaxScaler().fit_transform(X_verify)

In [33]:
print(trainx.shape)
trainx[0]

(15048, 49)


array([0.        , 0.666626  , 0.        , 0.        , 0.09207726,
       0.09207726, 0.01457519, 0.01457519, 0.        , 0.        ,
       0.52021664, 0.07686406, 0.07385634, 0.06909325, 0.05371942,
       0.05477189, 0.23205402, 0.        , 0.58604234, 0.        ,
       1.        , 1.        , 0.5       , 0.1969697 , 0.5011382 ,
       0.        , 1.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 1.        ,
       0.        , 1.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 1.        , 0.        ], dtype=float32)

In [34]:
temp = pd.DataFrame(Y_train)
temp.shape

(15048, 1)

## 先试试这样直接用SVM，因为之前使用VarianceThreshold挑选特征的时候train和verify两个上面挑选出来的特征不一样，这样就不太好搞啊。在train上挑选出来有6个特征，verify上挑选出来只有1个特征

In [35]:
def Linear_SVM(data_train, data_test, lable_train, lable_test):
    # 用网格搜索法来获取最优的C
    # 搜索最优的C值
    param_grid = {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['scale', 'auto']}
    clf = GridSearchCV(svm.SVC(degree=5, max_iter=10000), cv=3, param_grid=param_grid, refit=True, )
    clf.fit(data_train, lable_train)
    # 预测数据
    predict = clf.predict(data_test)
    # 生成准确率
    accuracy_rate = metrics.accuracy_score(lable_test, predict)
    print('精度为%s' % accuracy_rate)

In [36]:
Y_train = Y_train.astype('int')
Y_verify = Y_verify.astype('int')

In [37]:
Linear_SVM(trainx, verifyx, Y_train, Y_verify)

D:\Anaconda3\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver t

精度为1.0


In [38]:
print(trainx.shape, verifyx.shape)

(15048, 49) (30577, 49)


In [39]:
pca = PCA(n_components=10)
pca.fit(trainx)
trainx_t1 = pca.transform(trainx)
verifyx_t1 = pca.transform(verifyx)

In [40]:
Linear_SVM(trainx_t1, verifyx_t1, Y_train, Y_verify)

精度为1.0
